In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [2]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [3]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
new_model = "llama-2-7b-esc"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [5]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [6]:
training_params = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 15,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 10,
    per_device_eval_batch_size = 10,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "linear",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 20,
    logging_steps = 20,
    report_to="tensorboard", #"all"
    evaluation_strategy="steps",
    eval_steps=20  # 每5部驗證
)

In [7]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
    

print_trainable_parameters(model)

trainable params: 262410240 || all params: 3500412928 || trainable%: 7.496550989769399


In [12]:
def format_dialogue_prompt(messages):
    # 定義特殊標記
    INST_START, INST_END = "[INST]", "[/INST]"
    
    BOS, EOS = "<s>", "</s>"

    # 在對話開始處添加系統提示
    system_instruct = f'{BOS}{INST_START}'

    context = []
    context_cnt = 0  

    for message in messages:
        role = message['role']

        if context_cnt % 2 == 0 and role == 'user':
            content = message['content']
            context.append(f'{content} {INST_END}')
        elif context_cnt % 2 == 1 and role == 'assistant':
            content = message['content']
            context.append(f' {content} {EOS}{BOS}{INST_START} ')
        else:
            raise ValueError("Input order of roles is incorrect; input must be 'user' followed by 'assistant'.")

        context_cnt += 1  

    # 組合對話提示
    output = system_instruct + "".join(context)

    # 如果結尾不是assistant，返回完整的prompt
    if role != 'assistant':
        return output
    else:
        return output[:-len(BOS + INST_START)-1]

In [13]:
import pandas as pd

In [14]:
df = pd.read_excel('dataset.xlsx')

In [15]:
df.head()

,Unnamed: 0,year,class,prompt,response
0,1,2022,Hypertension,What is the new definition of hypertension in ...,Hypertension is now defined as a blood pressur...
1,2,2022,Hypertension,What blood pressure targets are set for hypert...,The universal blood pressure target recommende...
2,3,2022,Hypertension,Can you explain the 722 protocol for Home Bloo...,The 722 protocol involves taking duplicate BP ...
3,4,2022,Hypertension,What lifestyle changes are advised for managin...,Lifestyle modifications suggested include sodi...
4,5,2022,Hypertension,What are the first-line drugs recommended for ...,The recommended first-line antihypertensive dr...


In [16]:
# 先測試100個
import pandas as pd 
from sklearn.model_selection import train_test_split

df = pd.read_excel('dataset.xlsx').values[:]
qa_data = []
for _,_,_,question, answer in df:
    qa = [{'role':'user', 'content': f'{question}'}, {'role':'assistant', 'content': f'{answer}'}]
    qa_data.append(format_dialogue_prompt(qa))
    
x_train, x_valid = train_test_split(qa_data, train_size=0.9, random_state=46, shuffle=True)

In [17]:
train_dict_ = {'text':x_train}
df_train = pd.DataFrame(train_dict_)

In [18]:
train_dict_

{'text': ['<s>[INST]What is the significance of early mobilization in ACS management? [/INST] Early mobilization as part of the recovery process helps in assessing patient stability, reducing hospital stay, and improving overall recovery. </s>',
  '<s>[INST]What are the key messages regarding secondary hypertension? [/INST] Key messages include the importance of identifying underlying causes of secondary hypertension, such as kidney disease or hormonal disorders, which may require specific treatments. </s>',
  '<s>[INST]What is the importance of revascularization in ACS? [/INST] Revascularization, either by PCI or surgery, is crucial for restoring blood flow to the heart muscle affected by a blocked artery. </s>',
  '<s>[INST]What role do statins play in ACS management? [/INST] Statins are critical for lowering cholesterol levels, stabilizing plaque, and reducing the risk of subsequent cardiac events. </s>',
  '<s>[INST]What are the main challenges in managing CCS as identified in the 

In [19]:
df_train.to_csv('train.csv')

In [20]:
train_dataset = load_dataset('csv',data_files="train.csv", split = 'train')

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'text'],
    num_rows: 90
})

In [22]:
test_dict_ = {'text':x_valid}
df_text = pd.DataFrame(test_dict_)
df_text.to_csv('test.csv')
test_dataset = load_dataset('csv',data_files="test.csv", split = 'train')

Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
test_dataset

Dataset({
    features: ['Unnamed: 0', 'text'],
    num_rows: 10
})

In [23]:
###訓練之前的答案
prompt = "How does the guideline suggest using clinical risk scores for stroke prediction?"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe(f"[INST] {prompt} [/INST]", max_length = 200)
print(result[0]['generated_text']) # 輸出生成的文本

[INST] How does the guideline suggest using clinical risk scores for stroke prediction? [/INST]  The guideline suggests using clinical risk scores for stroke prediction by providing the following recommendations:
 Hinweis 1: Use clinical risk scores to estimate the probability of stroke in individuals with acute neurological symptoms.

* Clinical risk scores, such as the National Institutes of Health Stroke Scale (NIHSS) or the ABCD2 score, can estimate the probability of stroke in individuals with acute neurological symptoms.
* These scores take into account various factors, such as age, gender, history of stroke or TIA, and the presence of certain symptoms, to provide a quantitative estimate of stroke probability.
* Using these scores can help clinicians identify patients who are at high risk of stroke and prioritize them for urgent assessment and management.

Hint 2: Consider the use of clinical risk scores in addition to conventional stroke risk factors.

* Conventional stroke risk

In [24]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset = test_dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

/home/u8227385/.local/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/u8227385/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# 開始訓練模型
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:434: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
20,2.884700,2.083951
40,1.538600,1.390242
60,1.130400,1.237896


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:434: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:434: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain curr

TrainOutput(global_step=75, training_loss=1.6808369445800782, metrics={'train_runtime': 433.4633, 'train_samples_per_second': 3.114, 'train_steps_per_second': 0.173, 'total_flos': 2466826904371200.0, 'train_loss': 1.6808369445800782, 'epoch': 15.0})

In [31]:
###訓練之後的答案
prompt = "What future research directions does the guideline identify?"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe(f"[INST]{prompt}[/INST]", max_length = 400)
print(result[0]['generated_text']) # 輸出生成的文本

[INST]What future research directions does the guideline identify?[/INST] The guideline emphasizes the need for ongoing research to improve the diagnosis and management of atrial fibrillation, particularly in underserved populations and in the context of emerging technologies. obviously, the guideline highlights the importance of continued research to optimize the efficacy and safety of new anticoagulation strategies, including novel oral anticoagulants. 

The guideline also stresses the need for further studies to determine the optimal duration of anticoagulation therapy, particularly in patients with acute coronary syndromes.

In addition, the guideline acknowledges the growing interest in catheter ablation as a potential alternative to long-term anticoagulation therapy in select patient populations. The guideline encourages further research to compare the efficacy and safety of catheter ablation with continued anticoagulation therapy in various patient populations.

Overall, the gui

In [35]:
trainer.model.save_pretrained(new_model)

In [36]:

model_path = "/home/u8227385/PTT_Llama-2_fine-tune"  # 更改為您的路徑

# 以FP16重新載入模型並將其與LoRA權重合併
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [37]:

# 重新載入分詞器以進行保存
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 儲存合併後的模型
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)



('/home/u8227385/PTT_Llama-2_fine-tune/tokenizer_config.json',
 '/home/u8227385/PTT_Llama-2_fine-tune/special_tokens_map.json',
 '/home/u8227385/PTT_Llama-2_fine-tune/tokenizer.json')

In [38]:
# 下載新模型
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch


model_path = "/home/u8227385/PTT_Llama-2_fine-tune"  # 更改為您儲存模型的路徑

model = AutoModelForCausalLM.from_pretrained(model_path,
                         device_map="auto",
                         offload_folder="offload",
                         torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
os.getcwd()

'/home/u8227385'

In [40]:
prompt = "What is a large language model?"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text']) # 輸出生成的文本

/home/u8227385/.local/lib/python3.10/site-packages/transformers/models/open_llama/modeling_open_llama.py:42: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[INST] What is a large language model? [/INST]  A large language model is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. The model is designed to learn the patterns and structures of language, such as grammar, syntax, and semantics, by analyzing a large amount of text data.

The size of a large language model can vary, but it is typically measured in terms of the number of parameters or the amount of training data used to train the model. For example, a language model with 100 million parameters might be considered large, while one with 1 billion parameters might be considered enormous.

Some examples of large language models include:

1. BERT (Bidirectional Encoder Representations from Transformers): Developed by Google in 2018, BERT is a pre-trained language model that has been fine-tuned on a wide range of natural language processing (NLP) tasks, including question answe

In [42]:
prompt = "醫學系跟牙醫系誰比較好？"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text']) # 輸出生成的文本

[INST] 醫學系跟牙醫系誰比較好？ [/INST] 醫學系和牙醫系都是很好的學科，但是它們的背景和訓練有所不同。

醫學系是一個很具有挑戰性的學科，需要學生具備很強的學術基礎和實際醫療技能，以及具備廣泛的醫療知識和技能。醫學系的學生需要通過嚴苛的考試和實習，以確保他們能夠對於醫療護理和醫療科學的掌握有很強的基礎。

牙醫系則是一個很具有創新性和趣味的學科，需要學生具備很強的技術和實際技能，以及對於牙醫學的廣泛知識和技能。牙醫系的學生需要通過嚴苛的考試和實習，以確保他們能夠對於牙醫護理和牙醫科學的掌握有很強的基礎。

因此，醫學系和牙醫系的好壞差不多，都是�


In [43]:
prompt = "為什麼PTT這麼多人看棒球？"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text']) # 輸出生成的文本

[INST] 為什麼PTT這麼多人看棒球？ [/INST] 許多人看棒球的原因有多個，以下是一些可能的原因：

1. 棒球是台灣最受歡迎的運動，並且有許多優秀的選手，例如王建民、楊廣傑、楊顯煌等，這些選手的表現和個人特質吸引了許多人的追隨。
2. 棒球是一種需要團體合作的運動，因此團體合作的能力和團體精神具有很高的價值，這些價值可以帶給人們很大的感染和啟示。
3. 棒球是一種需要具有很高的體能和技術的運動，因此許多人對棒球有著崇敬和尊敬的態度，並且這些選手的努力和奮鬥具有很大的吸引力。
4. 棒球是一種需要具有很高的精神和決心的運動，因此許多人對棒球有著崇敬和尊敬的態度，並且這些選手的精神和決心具有很大的


In [38]:
def RLHF_loss(sentence_A, sentence_B):
    j = tokenizer(sentence_A+sentence_B, return_tensors="pt")
    k = tokenizer(sentence_B, return_tensors="pt")
    
    rewards_j = model(**j)[0]
    rewards_k = model(**k)[0]
    
    loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
    
    return loss

In [44]:
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=1e-4)

model.float().train()
def RLHF_loss(sentence_A, sentence_B):
    j = tokenizer(sentence_A, truncation=True, padding="longest", return_tensors='pt')
    k = tokenizer(sentence_B, truncation=True, padding="longest", return_tensors='pt')
    
    rewards_j = model(**j)[0]
    rewards_k = model(**k)[0]

    loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
    
    return loss
    
model.train()
loss = RLHF_loss(formatted_answer, decode_sentence[0])
loss.backward()
optimizer.step() 

RuntimeError: The size of tensor a (64) must match the size of tensor b (79) at non-singleton dimension 1

In [34]:
loss

NameError: name 'loss' is not defined

In [97]:
sentence_A = formatted_answer
sentence_B = decode_sentence[0]
tokenizer.truncation_side = "right" 
sentence_token = tokenizer([sentence_A,sentence_B], truncation=True,  padding = True,max_length = 128,return_tensors='pt')

A_dict = {'input_ids':sentence_token['input_ids'][0],'attention_mask':sentence_token['attention_mask'][0]}
B_dict = {'input_ids':sentence_token['input_ids'][1],'attention_mask':sentence_token['attention_mask'][1]}

rewards_j = model(A_dict)
#loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()

AttributeError: 'dict' object has no attribute 'shape'

In [114]:
A_dict = {'input_ids':sentence_token['input_ids'][0][None],'attention_mask':sentence_token['attention_mask'][0][None]}

rewards_j = model(**A_dict)

In [118]:
A_dict

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     1,     1,   518, 25580, 29962,
           3532, 14816, 29903,  6778,    13,  3057,   365, 26369,    13, 29966,
            829, 14816, 29903,  6778,    13,    13,   236,   137,   174, 31274,
          31185,   235,   186,   162,   234,   140,   156,   236,   137,   174,
          31185,   235,   173,   179, 31419,   235,   191,   134, 31076, 30882,
            518, 29914, 25580, 29962, 29871,   234,   140,   156,   236,   137,
            174, 30257,   234,   144,   181, 30753, 31721, 29871,     2]]),
 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1]])}

In [119]:
j

{'input_ids': tensor([[    1,     1,   518, 25580, 29962,  3532, 14816, 29903,  6778,    13,
          3057,   365, 26369,    13, 29966,   829, 14816, 29903,  6778,    13,
            13,   236,   137,   174, 31274, 31185,   235,   186,   162,   234,
           140,   156,   236,   137,   174, 31185,   235,   173,   179, 31419,
           235,   191,   134, 31076, 30882,   518, 29914, 25580, 29962, 29871,
           234,   140,   156,   236,   137,   174, 30257,   234,   144,   181,
         30753, 31721, 29871,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [115]:
rewards_j

CausalLMOutputWithPast(loss={'logits': tensor([[[-3.1270, -1.4300,  1.4025,  ..., -3.0038, -3.4245, -2.3205],
         [-3.2038, -1.5083,  1.4185,  ..., -3.0303, -3.4934, -2.3731],
         [-3.1429, -1.4212,  1.4134,  ..., -2.9934, -3.4374, -2.3246],
         ...,
         [-8.0971, -0.2395,  5.6717,  ...,  0.6724, -8.0394, -7.3654],
         [-5.0346,  1.4809, 13.4588,  ...,  3.1103, -5.6519, -3.9508],
         [-4.2991, -4.6206, 25.6700,  ..., -3.8327, -6.2832, -5.0528]]],
       grad_fn=<ToCopyBackward0>)}, logits=tensor([[[-3.1270, -1.4300,  1.4025,  ..., -3.0038, -3.4245, -2.3205],
         [-3.2038, -1.5083,  1.4185,  ..., -3.0303, -3.4934, -2.3731],
         [-3.1429, -1.4212,  1.4134,  ..., -2.9934, -3.4374, -2.3246],
         ...,
         [-8.0971, -0.2395,  5.6717,  ...,  0.6724, -8.0394, -7.3654],
         [-5.0346,  1.4809, 13.4588,  ...,  3.1103, -5.6519, -3.9508],
         [-4.2991, -4.6206, 25.6700,  ..., -3.8327, -6.2832, -5.0528]]],
       grad_fn=<ToCopyBackward0>),

In [116]:
j = tokenizer(sentence_A, truncation=True,  padding = True,max_length = 128,return_tensors='pt')
rewards_j = model(**j)
rewards_j

CausalLMOutputWithPast(loss={'logits': tensor([[[-13.1649,  -8.6807,   1.8199,  ...,  -3.7140,  -7.7171,  -8.0970],
         [-12.6191,  -8.3648,   2.1562,  ...,  -3.4286,  -7.4037,  -7.7408],
         [ -9.7628,  -3.3489,  -1.9629,  ...,  -2.2082,  -7.5787,  -3.5228],
         ...,
         [ -8.1117,  -0.2488,   5.8496,  ...,   0.8617,  -8.0798,  -7.3269],
         [ -5.1161,   1.5058,  13.5493,  ...,   3.2191,  -5.6692,  -4.0593],
         [ -4.3173,  -4.1605,  25.6111,  ...,  -3.6623,  -6.3189,  -5.1465]]],
       grad_fn=<ToCopyBackward0>)}, logits=tensor([[[-13.1649,  -8.6807,   1.8199,  ...,  -3.7140,  -7.7171,  -8.0970],
         [-12.6191,  -8.3648,   2.1562,  ...,  -3.4286,  -7.4037,  -7.7408],
         [ -9.7628,  -3.3489,  -1.9629,  ...,  -2.2082,  -7.5787,  -3.5228],
         ...,
         [ -8.1117,  -0.2488,   5.8496,  ...,   0.8617,  -8.0798,  -7.3269],
         [ -5.1161,   1.5058,  13.5493,  ...,   3.2191,  -5.6692,  -4.0593],
         [ -4.3173,  -4.1605,  25.6111,  

In [89]:
sentence_token = tokenizer([sentence_A,sentence_B], truncation=True,  padding = True,max_length = 128,return_tensors='pt')


In [92]:
sentence_token

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     1,     1,   518, 25580, 29962,
          3532, 14816, 29903,  6778,    13,  3057,   365, 26369,    13, 29966,
           829, 14816, 29903,  6778,    13,    13,   236,   137,   174, 31274,
         31185,   235,   186,   162,   234,   140,   156,   236,   137,   174,
         31185,   235,   173,   179, 31419,   235,   191,   134, 31076, 30882,
           518, 29914, 25580, 29962, 29871,   234,   140,   156,   236,   137,
           174, 30257,   234,   144,   181, 30753, 31721, 29871,     2],
        [    1,     1,     1, 29871,   518, 25580, 29962,  3532, 14816, 29903,
          6778,    13,  3057,   365, 26369,    13, 29966,   829, 14816, 29903,
          6778,    13,    13,   236,   137,   174, 31274, 31185,   235,   186,
           162,   234,   140,   156,   236,   137,   174, 31185,   235,   173,
           179, 31419,   235,   191,   134, 

In [93]:
sentence_token['input_ids'][0],sentence_token['attention_mask'][0]

(tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     1,     1,   518, 25580, 29962,
          3532, 14816, 29903,  6778,    13,  3057,   365, 26369,    13, 29966,
           829, 14816, 29903,  6778,    13,    13,   236,   137,   174, 31274,
         31185,   235,   186,   162,   234,   140,   156,   236,   137,   174,
         31185,   235,   173,   179, 31419,   235,   191,   134, 31076, 30882,
           518, 29914, 25580, 29962, 29871,   234,   140,   156,   236,   137,
           174, 30257,   234,   144,   181, 30753, 31721, 29871,     2]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]))

In [82]:
sentence_A+sentence_B

'<s>[INST] <<SYS>>\nTest LLM\n<</SYS>>\n\n醫學系跟牙醫系誰比較好？ [/INST] 牙醫大獲全勝 </s><s><s> [INST] <<SYS>>\nTest LLM\n<</SYS>>\n\n醫學系跟牙醫系誰比較好？ [/INST] 當然醫學系好啊 醫生很有錢 </s>'